# In this nodebook, we are going to cover some of the fundamental concepts of tensors using TensorFlow

More specifically are we are going to cover:
* Introduction to tensors
* Getting information from tensor
* Manipulating Tensors
* Tensors & NumPy
* Using @tf.function (a way to speed up regular Python functions)
* Using GPUs with TensorFlow (or TPUs)
* Exercises for yourself

### Introduction to Tensors

In [1]:
# Import TensorFlow
import tensorflow as tf

print(tf.__version__)

2024-01-05 16:21:24.872825: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-05 16:21:27.134885: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-05 16:21:27.134974: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-05 16:21:27.429136: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-05 16:21:28.163177: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-05 16:21:28.164690: I tensorflow/core/platform/cpu_feature_guard.cc:1

2.15.0


In [4]:
# Create tensors with tf.contants()
scalar = tf.constant(7)
scalar

<tf.Tensor: shape=(), dtype=int32, numpy=7>

In [5]:
# check for number of dimensions of a tensors (ndim stands for number of dimensions)
scalar.ndim

0

In [9]:
# create a vector
vector = tf.constant([10, 10])
vector

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([10, 10], dtype=int32)>

In [10]:
# check dimension of vector
vector.ndim

1

In [12]:
# create a matrix (has more than 1 dimension)
matrix = tf.constant([[10, 7],
                     [7, 10]])
matrix

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[10,  7],
       [ 7, 10]], dtype=int32)>

In [13]:
matrix.ndim

2

In [17]:
# create another matrix
another_matrix = tf.constant(
    [
        [4., 2., 7.],
        [3., 8., 10.],
        [6., 5., 9.]
    ],
    dtype=tf.float16)
another_matrix

<tf.Tensor: shape=(3, 3), dtype=float16, numpy=
array([[ 4.,  2.,  7.],
       [ 3.,  8., 10.],
       [ 6.,  5.,  9.]], dtype=float16)>

In [18]:
another_matrix.ndim

2

In [22]:
# let's create a tensor with 3 dimension
tensor = tf.constant(
    [
        [
            [1, 2],
            [3, 4]
        ],

        [
            [5, 6],
            [7, 8]
        ],

        [
            [9, 10],
            [11, 12]
        ]
    ])

tensor

<tf.Tensor: shape=(3, 2, 2), dtype=int32, numpy=
array([[[ 1,  2],
        [ 3,  4]],

       [[ 5,  6],
        [ 7,  8]],

       [[ 9, 10],
        [11, 12]]], dtype=int32)>

In [23]:
tensor.ndim

3

What we have created so far:

* Scalar: A single number
* Vector: A number with direction (e.g. wind speed and direction)
* Matrix: A 2 dimensional array of numbers
* Tensor: A n-dimensional array of numbers (where n can be any number, 0-dimensional is a scalar, 1-dimensional is a vector)

Create tensors with tf.Variable

In [4]:
# create the same tensor with tf.Variable() as above
changeable_tensor = tf.Variable([10, 7])
unchangeable_tensor = tf.constant([10, 7])

changeable_tensor, unchangeable_tensor

(<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([10,  7], dtype=int32)>,
 <tf.Tensor: shape=(2,), dtype=int32, numpy=array([10,  7], dtype=int32)>)

In [6]:
# let's try to change one of the elements in changable tensor
changeable_tensor[0] = 7
changeable_tensor

TypeError: 'ResourceVariable' object does not support item assignment

In [7]:
# How about we try .assign()
changeable_tensor[0].assign(3)
changeable_tensor

<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([3, 7], dtype=int32)>

In [8]:
# Lets try to change unchangable_tensor
unchangeable_tensor[0].assign(3)
unchangeable_tensor

AttributeError: 'tensorflow.python.framework.ops.EagerTensor' object has no attribute 'assign'

### Creating Random Tensors

Random tensors are tensors of some arbitary size filled with random numbers

In [5]:
# Create two random (but the same) tensors
random_1 = tf.random.Generator.from_seed(42) # set seed for reproducability
random_1 = random_1.normal(shape=(3,2))
random_1

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[-0.7565803 , -0.06854702],
       [ 0.07595026, -1.2573844 ],
       [-0.23193763, -1.8107855 ]], dtype=float32)>

In [6]:
random_2 = tf.random.Generator.from_seed(42)
random_2 = random_2.normal(shape=(3,2))
random_2

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[-0.7565803 , -0.06854702],
       [ 0.07595026, -1.2573844 ],
       [-0.23193763, -1.8107855 ]], dtype=float32)>

In [7]:
# Are they equal?
random_1, random_2, random_1 == random_2

(<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[-0.7565803 , -0.06854702],
        [ 0.07595026, -1.2573844 ],
        [-0.23193763, -1.8107855 ]], dtype=float32)>,
 <tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[-0.7565803 , -0.06854702],
        [ 0.07595026, -1.2573844 ],
        [-0.23193763, -1.8107855 ]], dtype=float32)>,
 <tf.Tensor: shape=(3, 2), dtype=bool, numpy=
 array([[ True,  True],
        [ True,  True],
        [ True,  True]])>)

### Shuffle the order of elements in tensor

In [15]:
# Shuffle a tensor (valuable when you want to shuffle your data so that, inhirent order does not efffect learning)
not_shuffled = tf.constant(
    [
        [10, 7],
        [3, 4],
        [2, 5]
    ]
)

# shuffle our non_shuffled tensor
shuffled = tf.random.shuffle(not_shuffled)
shuffled

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[10,  7],
       [ 2,  5],
       [ 3,  4]], dtype=int32)>

In [58]:
# experementing on seed
tf.random.set_seed(55)
tf.random.shuffle(not_shuffled, seed=55)

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[ 2,  5],
       [10,  7],
       [ 3,  4]], dtype=int32)>